In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
from time import sleep # background color fade
import math # calculo com sin e cos

glfw.init()

glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(500, 500, "Teste", None, None)
glfw.make_context_current(window)

# keyboard/mouse event listeners
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')

glfw.set_key_callback(window,key_event)
glfw.set_mouse_button_callback(window,mouse_event)

# codigos do vertex e do fragment shader no pipeline
vertex_code = """
    attribute vec2 position;
    void main(){
        gl_Position = vec4(position,0.0,1.0);
    }
    """
fragment_code = """
        uniform vec4 color;

        void main() {
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

# calculo dos vertices do circulo
num_vertices = 64 # define a "qualidade" do circulo
pi = 3.14
counter = 0
radius = 1.0
vertices = np.zeros(num_vertices + 11, [("position", np.float32, 2)])

angle = 0.0
for counter in range(num_vertices):
    angle += 2*pi/num_vertices 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter] = [x,y]

#print(vertices) # debug

# colocando os vertices do quadrado
vertices[num_vertices] = [+0.5, -0.5]
vertices[num_vertices + 1] = [+0.5, +0.5]
vertices[num_vertices + 2] = [-0.5, -0.5]
vertices[num_vertices + 3] = [-0.5, +0.5]
#print(vertices) # debug

# vertices do triangulo
vertices[num_vertices + 4] = [+0.5, -0.5]
vertices[num_vertices + 5] = [-0.5, -0.5]
vertices[num_vertices + 6] = [-0.0, +0.5]
#print(vertices) # debug

# vertices da linha
vertices[num_vertices + 7] = [-1.0, -1.0]
vertices[num_vertices + 8] = [+1.0, +1.0]
#print(vertices) # debug

# vertices dos pontos
vertices[num_vertices + 9] = [+0.5, -0.25]
vertices[num_vertices + 10] = [-0.5, +0.25]
#print(vertices) # debug

# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

R = 1.0
G = 0.0
B = 0.0
switcher = 0

glfw.show_window(window)

while not glfw.window_should_close(window):

    #efeito RGB em loop
    if B < 0.03:
        switcher = 0
    if B > 0.99:
        switcher = 1

    sleep(0.03)
    
    if switcher == 0:
        B = B + 0.03
        G = G + 0.01
        R = R + 0.01
    else:
        B = B - 0.03
        G = G - 0.01
        R = R - 0.01
    
    # funcao interna do glfw para gerenciar eventos de mouse, teclado, etc
    glfw.poll_events() 

    #glPolygonMode(GL_FRONT_AND_BACK, GL_LINE) ## ative esse comando para enxergar os triângulos
    
    # limpa a cor de fundo da janela e preenche com outra no sistema RGBA
    glClear(GL_COLOR_BUFFER_BIT)
    
    # definindo a cor da janela      
    glClearColor(R, G, B, 1.0)
    
    loc_color = glGetUniformLocation(program, "color")
    glUniform4f(loc_color, R + 0.2, G + 0.2, B + 0.2, 1.0); # modificando a cor do circulo
    glDrawArrays(GL_TRIANGLE_FAN, 0, num_vertices)
    
    glUniform4f(loc_color, R + 0.4, G + 0.4, B + 0.4, 1.0); # modificando a cor do quadrado
    glDrawArrays(GL_TRIANGLE_STRIP, num_vertices, 4)
    
    glUniform4f(loc_color, R + 0.7, B + 0.7, R + 0.7, 1.0); # modificando a cor do triangulo
    glDrawArrays(GL_TRIANGLES, num_vertices + 4, 3)
    
    glUniform4f(loc_color, 0, 0, 0, 1.0); # modificando a cor da linha
    glDrawArrays(GL_LINES, num_vertices + 7, 2)
    
    glUniform4f(loc_color, 0, 0, 0, 1.0); # modificando a cor dos pontos
    glDrawArrays(GL_POINTS, num_vertices + 9, 2)
    
    # linha extra ligando os dois pontos (amarela)
    glUniform4f(loc_color, 1, 1, 0, 1.0);
    glDrawArrays(GL_LINES, num_vertices + 9, 2)
    
    # gerencia troca de dados entre janela e o OpenGL
    glfw.swap_buffers(window)
    
glfw.terminate()